In [6]:
# Python lib
import asyncio
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from typing import Dict, Any, Tuple
import venv

# Autogen-0.4
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.agents._code_executor_agent import CodeExecutorAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Local
from prompts import cleaning_reasoning_prompt, cleaning_checking_prompt
from utils import CopyFile, LoadDataset, GetDatasetProfile, Spinner

# Only edit here AND filepath under if __name__ == "__main__":
reasoning_model = "qwen2.5:32b-instruct-q8_0"
coding_model = "qwen2.5-coder:32b-instruct-q8_0"

# Common config
llm_base_url = "http://34.204.63.234:11434/v1"
api_key = "none"
capabilities =  {
        "vision": False,
        "function_calling": False,
        "json_output": False
    }

#######################################################################
#   !!! DONT EDIT BELOW EXCEPT FOR if __name__ == "__main__":   !!!   #
#######################################################################

# Reasoning Model Configuration
instruct_client_config = OpenAIChatCompletionClient(
    model=reasoning_model,
    base_url=llm_base_url,
    api_key=api_key,
    model_capabilities=capabilities
)

# Coding Model Configuration
code_client_config = OpenAIChatCompletionClient(
    model=coding_model,
    base_url=llm_base_url,
    api_key=api_key,
    model_capabilities=capabilities
)

In [7]:
async def create_cleaning_agents(filepath: Path) -> Tuple[AssistantAgent, AssistantAgent, CodeExecutorAgent, AssistantAgent]:
    
    # tqdm progress bar
    with tqdm(total=5,
             desc="Creating cleaning team agents",
             bar_format='{desc:>30}{postfix: <1} {bar}|{n_fmt:>4}/{total_fmt:<4}',
             colour='green') as pbar:
        
        ### 1.1 Cleaning reasoning agent - 1
        async def create_cleaning_reasoning_agent():
            cleaning_reasoning_agent = AssistantAgent(
                name="cleaning_reasoning_agent",
                is_termination_msg='',
                model_client=instruct_client_config,
                system_message=cleaning_reasoning_prompt(),
            )
            pbar.update(1)
            return cleaning_reasoning_agent
        
        async def create_cleaning_checker():
            cleaning_checker = AssistantAgent(
                name="cleaning_checker",
                model_client=instruct_client_config,
                system_message=cleaning_checking_prompt()
            )
            pbar.update(1)
            return cleaning_checker
        
        list_of_cleaning_agents = await asyncio.gather(
            create_cleaning_reasoning_agent(),
            create_cleaning_checker(),
        )
        return list_of_cleaning_agents

In [8]:
async def run_cleaning_pipeline(df: pd.DataFrame, data_dict: Dict[str, Any], filepath: Path) -> pd.DataFrame:
    """
    Run the complete data cleaning and transformation pipeline
    """
    try:
        # cleaning_reasoning_1, cleaning_reason_2, cleaning_coding, code_executor, code_checker
        cleaning_team = await create_cleaning_agents(filepath=filepath)
        cleaning_team = [cleaning_team[0], cleaning_team[1]]
        
        # Setup termination conditions
        text_term = TextMentionTermination("TERMINATE") # If the text "TERMINATE" is mentioned - text
        round_term = MaxMessageTermination(5) #n          # If max message reaches n round - round
        termination = text_term | round_term            # text OR n round is met

        # First phase: Data Cleaning (Reasoning)
        cleaning_team_chat = RoundRobinGroupChat(
            cleaning_team,
            termination_condition=termination,
        )
        
        # prompt imported from .prompts/
        cleaning_task = cleaning_reasoning_prompt() + f"""
<dataset_location>
    {str(filepath)}
</dataset_location>

<data_dictionary>
    {data_dict}
</data_dictionary>
"""

        # A loading spinner to know if the code is frozen or not
        await Spinner.async_with_spinner(
            message="Loading: ",
            style="braille",
            console_class=Console,
            coroutine=cleaning_team_chat.run_stream(task=cleaning_task)
        )
    except Exception as e:
        raise Exception(f"An error occurred: {str(e)}")


In [9]:
with tqdm(total=3,
          desc="Preparing dataset",
          bar_format='{desc:>30}{postfix: <1} {bar}|{n_fmt:>4}/{total_fmt:<4}',
          colour='green') as pbar:
    # Edit file path
    test_file = Path("../sheets/credit_card_transactions.csv")
    pbar.update(1)
    df = LoadDataset(test_file)
    pbar.update(1)
    # Custom function in utils/ to get data dict in md format
    initial_profile = GetDatasetProfile(df, output_format="markdown")
    pbar.update(1)



         Processing 24 columns  ██████████|  24/24  
             Preparing dataset  ██████████|   3/3   


In [10]:
await run_cleaning_pipeline(df, initial_profile, test_file)

 Creating cleaning team agents  ████      |   2/5   

⠋ Loading: 

---------- user ----------
<Data Cleaning Planner>

<purpose>
    Provide detailed and actionable data cleaning recommendations with domain-aware considerations for each column.
</purpose>

<instructions>
    1. Review the provided data dictionary thoroughly.
    2. Analyze data types and their real-world significance.
    3. Identify potential data quality issues without limiting to predefined categories, considering:
       - Relationships between columns (e.g., geographic data, dates)
       - Data type constraints
       - Domain-specific valid ranges
       - Business logic dependencies
       - Any other anomalies or irregularities present in the data
    4. FOR EACH IDENTIFIED ISSUE, PROVIDE A SINGLE, EXACT DATA CLEANING TECHNIQUE TO BE USED.
       - **Do NOT present multiple options or suggest evaluations.**
       - Use concrete methods (e.g., "Apply the Z-score method to detect and remove outliers beyond 3 standard deviations using NumPy and Pandas").
       - Ensure the tec